In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import os
import cv2
from tensorflow import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras import Sequential,optimizers,models
from tensorflow.keras.layers import Dropout,Input,Conv2D, GlobalAvgPool2D, BatchNormalization, Dense,AveragePooling2D,MaxPool2D,ReLU,add
import warnings
warnings.filterwarnings("ignore")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
model = keras.models.load_model('/home/kesci/work/eff_b5_other/model_weights03-0.79.h5')

In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('work/test', target_size=(224, 224),
batch_size=1,class_mode='categorical', shuffle=False,)

Found 72778 images belonging to 1 classes.


In [4]:
# test_generator.reset()
pred = model.predict_generator(test_generator, verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
72778/72778 [==============================] - 1287s 18ms/step


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, fill_mode='wrap')
train_generator = train_datagen.flow_from_directory('work/data', target_size=(224, 224), batch_size=96)

Found 143667 images belonging to 3 classes.


In [7]:
labels = (train_generator.class_indices)
label = dict((v,k) for k,v in labels.items())

In [10]:
label

{0: '0', 1: '1', 2: '2'}

In [11]:
# predicted_class_indices = np.argmax(pred, axis=1)
# predictions = [label[i] for i in predicted_class_indices]
filenames = test_generator.filenames

In [12]:
filenames = [i.split('/')[-1] for i in filenames]
# predictions = [int(i) for i in predictions]

In [13]:
result = pd.DataFrame(filenames,columns=['id'])
# result['weather'] = predictions

In [19]:
result['label_0'] = pred[:,0]
result['label_1'] = pred[:,1]
result['label_2'] = pred[:,2]

In [21]:
result.to_csv('work/b5_result.csv',index=0)

In [22]:
result

,id,label_0,label_1,label_2
0,100017842.jpg,0.699524,0.267646,0.032829
1,100057293.jpg,0.977726,0.004869,0.017405
2,1000599894.jpg,0.854945,0.096832,0.048224
3,1000600110.jpg,0.989104,0.006694,0.004202
4,1000974891.jpg,0.000258,0.952444,0.047297
...,...,...,...,...
72773,99997209972.jpg,0.000018,0.000220,0.999761
72774,99997809978.jpg,0.004195,0.106720,0.889085
72775,9999800998.jpg,0.023226,0.039805,0.936970
72776,99998609986.jpg,0.000734,0.023656,0.975610


In [11]:
result['weather'].value_counts()

2    33334
0    23739
1    15705
Name: weather, dtype: int64

In [14]:
result['weather'].value_counts()

2    33761
0    23764
1    15253
Name: weather, dtype: int64

In [7]:
# df1 = pd.read_csv('work/result.csv')
# df1.columns = ['id','label']
# df1 = pd.get_dummies(df1,columns=['label'])
# df2 = pd.read_csv('work/B7.csv')
# df2.columns = ['id','label']
# df2 = pd.get_dummies(df2,columns=['label'])
# df3 = pd.read_csv('work/model_net.csv')
# df3.columns = ['id','label']
# df3 = pd.get_dummies(df3,columns=['label'])

In [8]:
df3['weather'].value_counts()

2    31665
0    26071
1    15042
Name: weather, dtype: int64

In [3]:
df1 = df1.sort_values(['id']).reset_index(drop=True)
df2 = df2.sort_values(['id']).reset_index(drop=True)
df3 = df3.sort_values(['id']).reset_index(drop=True)

In [4]:
df1['label_0'] = 0.34 * df1['label_0'] + 0.36 * df2['label_0'] + 0.3 * df3['label_0']
df1['label_1'] = 0.34 * df1['label_1'] + 0.36 * df2['label_1'] + 0.3 * df3['label_1']
df1['label_2'] = 0.34 * df1['label_2'] + 0.36 * df2['label_2'] + 0.3 * df3['label_2']

In [8]:
label = np.array(df1[['label_0','label_1','label_2']])
df1['weather'] = label.argmax(1)

In [10]:
df1[['id','weather']].to_csv('work/result_add.csv',index=0)

In [11]:
df1['weather'].value_counts()

2    32849
0    24195
1    15734
Name: weather, dtype: int64

In [12]:
!wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v4/kesci_submit&&chmod +x kesci_submit

2020-10-24 14:40:24 URL:https://cdn.kesci.com/submit_tool/v4/kesci_submit [7357446/7357446] -> "kesci_submit" [1]


In [13]:
!./kesci_submit -token 7a503d402d41caa3 -file work/result_add.csv

Kesci Submit Tool 4.0.0

> 已验证Token
> 提交文件 work/result_add.csv (1204.74 KiB), Target Qiniu
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
